In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import recall_score, f1_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)


df = pd.read_csv("https://github.com/oppenheimer3/Algeria_wildfires_prediction/raw/master/datasets/total_dataset.csv").drop(columns=['Unnamed: 0'])

scaler = StandardScaler()
df_numeric = df.drop(columns=['date', 'Province', 'Town', 'fire', 'longitude', 'latitude'])
df_normalized = pd.DataFrame(scaler.fit_transform(df_numeric), columns=df_numeric.columns)
df_normalized[['date', 'Province', 'Town', 'fire', 'longitude', 'latitude']] = df[['date', 'Province', 'Town', 'fire', 'longitude', 'latitude']]
df = df_normalized

df['year'] = pd.to_datetime(df['date']).dt.year
df_train = df[(df['Province']!='Bejaia') & (df['Town']!='El Kseur')].drop(columns=['year', 'Province', 'Town', 'date', 'longitude', 'latitude'])
df_test = df[(df['Province']=='Bejaia') & (df['Town']=='El Kseur')].drop(columns=['year', 'Province', 'Town', 'date', 'longitude', 'latitude'])

x_train = df_train.drop(columns = ['fire', 'MaxFRP'])
y_train = df_train['fire']
x_test = df_test.drop(columns = ['fire', 'MaxFRP'])
y_test = df_test['fire']

In [2]:
x_train = torch.tensor(x_train.values, dtype= torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
x_test = torch.tensor(x_test.values, dtype= torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)



In [3]:
train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

In [4]:


class Classifier(nn.Module):
  def __init__(self, hidden_layers):
    super().__init__()
    self.fc1 = nn.Linear(76, hidden_layers)
    self.fc2 = nn.Linear(hidden_layers,1)
    self.relu = nn.ReLU()
    self.sig = nn.Sigmoid()

  def forward(self, x):
    x = self.relu(self.fc1(x))
    o = self.sig(self.fc2(x))
    return o
  def train(self, optimizer, criterion, epochs, train_loader, x_val, y_val):
    t_lst = []
    v_lst = []
    for e in range(epochs):
      for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        predictions = self(inputs)
        loss = criterion(predictions, labels.view(-1,1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        val_loss = criterion(self(x_val.to(device)), y_val.view(-1,1).to(device))
        test_loss = criterion(self(x_test.to(device)), y_test.view(-1,1).to(device))
      t_lst.append(loss.item())
      v_lst.append(val_loss.item())
      print(e,loss.item(), val_loss.item())
      if val_loss<0.18:
        break

lr = 0.001
epochs = 100

model = Classifier(550).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)


In [5]:
model.train(optimizer, criterion, epochs, train_loader, x_val, y_val)

0 0.24441388249397278 0.3051410913467407
1 0.19055618345737457 0.254477858543396
2 0.1958860605955124 0.2324950098991394
3 0.13792069256305695 0.22306133806705475
4 0.09440300613641739 0.21480432152748108
5 0.2699992060661316 0.20645520091056824
6 0.2425108551979065 0.20323893427848816
7 0.1750408411026001 0.19751636683940887
8 0.23619508743286133 0.204312264919281
9 0.16958507895469666 0.19797886908054352
10 0.1158975288271904 0.1944839507341385
11 0.17725668847560883 0.19592726230621338
12 0.2042769342660904 0.18774078786373138
13 0.1015864908695221 0.1909448206424713
14 0.03888075053691864 0.19301515817642212
15 0.22126857936382294 0.18911811709403992
16 0.09481407701969147 0.18828184902668
17 0.1554538458585739 0.18493282794952393
18 0.2695687711238861 0.1844642609357834
19 0.11581581830978394 0.1858254075050354
20 0.13025833666324615 0.18124671280384064
21 0.1093854233622551 0.19189579784870148
22 0.19187115132808685 0.18749195337295532
23 0.06746426224708557 0.1860731840133667
24

In [6]:
criterion(model(x_test.to(device)), y_test.view(-1,1).to(device))

tensor(0.3040, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)

In [8]:
df_test = df[(df['Province']=='Bejaia') & (df['Town']=='El Kseur')]
df_test['prediction'] = model(x_test.to(device)).detach().cpu()
dates = ['2023-07-22', '2023-07-23', '2023-07-24', '2023-07-25']
df = df_test[['longitude', 'latitude', 'fire', 'prediction']]

<ipython-input-8-d24d124e3c48>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['prediction'] = model(x_test.to(device)).detach().cpu()


In [10]:
import geemap
import ee

ee.Authenticate()
ee.Initialize()

fc_lst = [ee.Feature(ee.Geometry.Point(row['longitude'], row['latitude']), {'prediction':row['prediction'], 'fire':row['fire']}) for i, row in df.iterrows() ]
fc = ee.FeatureCollection(fc_lst)
p_image = fc.reduceToImage(properties=['prediction'], reducer=ee.Reducer.first())
p_image = p_image.reproject('EPSG:3035', None, 1000)
r_image = fc.reduceToImage(properties=['fire'], reducer=ee.Reducer.first())
r_image = r_image.reproject('EPSG:3035', None, 1000)

Map = geemap.Map(center=[ 36.58,4.20], zoom=10)

Map.addLayer(r_image, {'min': 0, 'max': 1, 'palette': ['green', 'yellow', 'red']}, 'real')
Map.addLayer(p_image, {'min': 0, 'max': 1, 'palette': ['green', 'yellow', 'red']}, 'prediction')

Map.addLayerControl()
Map


To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=ksCY6mUy3_dmdzpprNKgPekHnXDbk_ff4ier1z2ztgw&tc=CS1KwRhun4FEihxhSQHzkBL532Gd9Ejg-1O84_lXkeQ&cc=4UpWXdubTYo50whQZSnzVhXBSP_ZVQqmHFSX6gakNWM

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXl3kVcG8sfeXgzhV01scvCZnrqwqsXkUtuKFpBj9og3MLiWZ5APKmw

Successfully saved authorization token.


Map(center=[36.58, 4.2], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…